In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, fcluster
import matplotlib.pyplot as plt
from tqdm import tqdm
from collections import deque
from scipy.cluster.hierarchy import linkage, fcluster
import scanpy as sc
import pandas as pd
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import squidpy as sq
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from threadpoolctl import threadpool_limits, threadpool_info
threadpool_limits(limits=8)
import itertools
from collections import deque
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.utils import resample
from matplotlib.backends.backend_pdf import PdfPages
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from sklearn.cluster import KMeans
from threadpoolctl import threadpool_limits, threadpool_info
threadpool_limits(limits=8)
import backSPIN
from sklearn.preprocessing import StandardScaler

In [ ]:
# do the feature selection, if needed
# the features we selected are based on:
# a) Moran's I (vs salt-and-pepper lipids, white noise-like)
# b) variance vs mean of section-wise variances

data = pd.read_hdf("/data/luca/lipidatlas/ManuscriptAnalysisRound2/20240724_featsel_lba.h5", key="table")
data = data.loc[(data['Section'] != 6) & (data['Section'] != 10) & (data['Section'] != 13) & (data['Section'] != 20) & (data['Section'] != 25),:]

coordinates = data[['Section', 'zccf', 'yccf', 'xccf']]
coordinates['Section'] = coordinates['Section'].astype(int)

data = data.iloc[:,20:210]
spatvar = np.load("/data/luca/lipidatlas/ManuscriptAnalysis/feature_selected_peaks-Copy1.npy", allow_pickle=True)

data = data.loc[:, np.intersect1d(data.columns, spatvar.astype(float))]
data

In [ ]:
# use NMF to decompose the data into factors; it works well with exp but not log data with current settings

from sklearn.decomposition import NMF
from threadpoolctl import threadpool_limits, threadpool_info
threadpool_limits(limits=8)
import os
import pickle
import uuid

os.environ['OMP_NUM_THREADS'] = '6'

nmfdfs = []

# specify the expected number of factors
N_factors = 15 #40 
num_iterations = 10 

for _ in tqdm(range(num_iterations)):
    drop_fraction = np.random.uniform(0.05, 0.30)
    columns_to_keep = np.random.choice(data.columns, 
                                       size=int(len(data.columns) * (1 - drop_fraction)), 
                                       replace=False)
    datads = data[columns_to_keep]
    
    iteration_id = str(uuid.uuid4())
    with open(f'kept_columns_{iteration_id}.pkl', 'wb') as f:
        pickle.dump(columns_to_keep, f)
    
    nmf = NMF(n_components=N_factors, init='random', random_state=230598)
    nmf_result = nmf.fit_transform(datads-np.min(datads)+1e-7)
    nmfdf = pd.DataFrame(nmf_result, index=datads.index)
    
    nmfdf.to_parquet(f'nmf_result_{iteration_id}.parquet')
    
    nmfdfs.append(nmfdf)

nmfdfs[0]

In [ ]:
# the baseline

from sklearn.decomposition import NMF
from threadpoolctl import threadpool_limits, threadpool_info
threadpool_limits(limits=8)
import os
os.environ['OMP_NUM_THREADS'] = '6'

# specify the expected number of factors
N_factors = 15 #40 

nmf = NMF(n_components=N_factors, init='random', random_state=230598)

nmf_result = nmf.fit_transform(data-np.min(data)+1e-7)

nmfdf = pd.DataFrame(nmf_result, index=data.index)

nmfdf

In [ ]:
mnn_corrected_nmf = np.load("nmf_result_corrected.npy")

In [ ]:
import matplotlib.pyplot as plt

n_rows = 15 
n_cols = 1 
fig, axes = plt.subplots(n_rows, n_cols, figsize=(2, 15))

axes = axes.flatten()

for xxx, ax in enumerate(axes):
    ax.scatter(nmfdf.iloc[:, xxx][::10], mnn_corrected_nmf[:, xxx][::10], 
               s=0.000005, alpha=0.5, c=mnn_corrected_nmf[:, xxx][::10], cmap="PuOr", rasterized=True)

    ax.grid(False)
    ax.set_xticks([])  # Remove x-axis ticks
    ax.set_yticks([])  # Remove y-axis ticks
    ax.set_xticklabels([])  # Remove x-axis tick labels
    ax.set_yticklabels([])  # Remove y-axis tick labels
    ax.spines['top'].set_visible(False)  # Remove top spine
    ax.spines['right'].set_visible(False)  # Remove right spine
    ax.spines['left'].set_visible(False)  # Remove left spine
    ax.spines['bottom'].set_visible(False)  # Remove bottom spine

plt.tight_layout()
plt.show()


In [ ]:
downsamp_nmf = nmfdf[::20]
downsamp_mnn_nmf = mnn_corrected_nmf[::20]

In [ ]:
means = np.mean(downsamp_nmf, axis=0)
stds = np.std(downsamp_nmf, axis=0)
downsamp_nmf = (downsamp_nmf - means) / stds

In [ ]:
means = np.mean(downsamp_mnn_nmf, axis=0)
stds = np.std(downsamp_mnn_nmf, axis=0)
downsamp_mnn_nmf = (downsamp_mnn_nmf - means) / stds

In [ ]:
## do a tSNE to see how the clustering method behaves globally as we go

from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization
import numpy as np

x_train = downsamp_nmf.values

affinities_train = affinity.PerplexityBasedNN(
    x_train,
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
)


init_train = x_train[:,[0,5]] # initialize with two uncorrelated NMFs, note this affects results a bit

embedding_train = TSNEEmbedding(
    init_train,
    affinities_train,
    negative_gradient_method="fft",
    n_jobs=8,
    verbose=True,
)

embedding_train_1 = embedding_train.optimize(n_iter=500, exaggeration=1.2) ########## parameters...

In [ ]:
plt.scatter(embedding_train_1[:,0], embedding_train_1[:,1],c=coordinates['Section'][::20].astype("category").cat.codes, cmap="nipy_spectral", s=0.1, alpha=0.5, rasterized=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have already imported necessary libraries and have the data ready

# Get unique sections
unique_sections = coordinates['Section'].unique()

# Create a figure with subplots for each unique section
fig, axs = plt.subplots(nrows=(len(unique_sections)+1)//2, ncols=2, figsize=(20, 5*len(unique_sections)//2))
axs = axs.flatten()  # Flatten the 2D array of axes for easy indexing

for i, section in enumerate(unique_sections):
    # Create a mask for the current section
    mask = coordinates['Section'][::20] == section
    
    # Plot all points in gray
    axs[i].scatter(embedding_train_1[:,0], embedding_train_1[:,1], 
                   c='gray', s=0.1, alpha=0.5, rasterized=True)
    
    # Plot the current section in red
    axs[i].scatter(embedding_train_1[mask,0], embedding_train_1[mask,1], 
                   c='red', s=0.1, alpha=0.5, rasterized=True)
    
    axs[i].set_title(f'Section: {section}')
    axs[i].set_xlabel('Dimension 1')
    axs[i].set_ylabel('Dimension 2')

# Remove any unused subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

In [ ]:
## do a tSNE to see how the clustering method behaves globally as we go

from openTSNE import TSNEEmbedding
from openTSNE import affinity
from openTSNE import initialization
import numpy as np

x_train = downsamp_mnn_nmf

affinities_train = affinity.PerplexityBasedNN(
    x_train,
    perplexity=30,
    metric="euclidean",
    n_jobs=8,
    random_state=42,
    verbose=True,
)


init_train = x_train[:,[0,5]] # initialize with two uncorrelated NMFs, note this affects results a bit

embedding_train = TSNEEmbedding(
    init_train,
    affinities_train,
    negative_gradient_method="fft",
    n_jobs=8,
    verbose=True,
)

embedding_train_2 = embedding_train.optimize(n_iter=500, exaggeration=1.2) ########## parameters...

In [ ]:
plt.scatter(embedding_train_2[:,0], embedding_train_2[:,1],c=coordinates['Section'][::20].astype("category").cat.codes, cmap="nipy_spectral", s=0.1, alpha=0.5, rasterized=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have already imported necessary libraries and have the data ready

# Get unique sections
unique_sections = coordinates['Section'].unique()

# Create a figure with subplots for each unique section
fig, axs = plt.subplots(nrows=(len(unique_sections)+1)//2, ncols=2, figsize=(20, 5*len(unique_sections)//2))
axs = axs.flatten()  # Flatten the 2D array of axes for easy indexing

for i, section in enumerate(unique_sections):
    # Create a mask for the current section
    mask = coordinates['Section'][::20] == section
    
    # Plot all points in gray
    axs[i].scatter(embedding_train_2[:,0], embedding_train_2[:,1], 
                   c='gray', s=0.1, alpha=0.5, rasterized=True)
    
    # Plot the current section in red
    axs[i].scatter(embedding_train_2[mask,0], embedding_train_2[mask,1], 
                   c='red', s=0.1, alpha=0.5, rasterized=True)
    
    axs[i].set_title(f'Section: {section}')
    axs[i].set_xlabel('Dimension 1')
    axs[i].set_ylabel('Dimension 2')

# Remove any unused subplots
for j in range(i+1, len(axs)):
    fig.delaxes(axs[j])

plt.tight_layout()
plt.show()

In [ ]:
ref_nmf = nmfdf

In [ ]:
pairwise_correlations_with_ref = []
maxs = []

from tqdm import tqdm

for df in tqdm(nmfdfs):
    correlation_matrix = pd.DataFrame(np.corrcoef(df.T, ref_nmf.T)[:len(df.columns), len(df.columns):])
    pairwise_correlations_with_ref.append(correlation_matrix)
    maxs.append(correlation_matrix.max())
    
maxs

In [ ]:
maxs = pd.DataFrame(maxs)

In [ ]:
maxs = maxs.T

In [ ]:
plt.imshow(maxs,cmap="cividis")
plt.colorbar()

In [ ]:
import os
import pickle

# List to store the contents of each .pkl file
kept_columns_list = []

# Directory where your files are stored (replace '.' with the path to your folder if needed)
directory = '.'

# Get all files starting with "kept_columns"
iteration_files = [f for f in os.listdir(directory) if f.startswith('kept_columns') and f.endswith('.pkl')]

# Iterate over each file and load the pickle contents
for file_name in iteration_files:
    file_path = os.path.join(directory, file_name)
    with open(file_path, 'rb') as f:
        columns_to_keep = pickle.load(f)
        kept_columns_list.append(len(columns_to_keep))

kept_columns_list

In [ ]:
len_kept_columns_list = kept_columns_list

In [ ]:
len_kept_columns_list = [str(len_value) for len_value in len_kept_columns_list]  # Convert entries to strings if needed

plt.imshow(maxs, cmap="cividis")
plt.colorbar()

plt.yticks(ticks=range(15), labels=[f"NMF{i+1}" for i in range(15)])

plt.xticks(ticks=range(len(len_kept_columns_list)), labels=[f"{entry}/108" for entry in len_kept_columns_list], rotation=90)

plt.show()

In [ ]:
#clustering_robn1.ipynb
#checks_on_clustering.ipynb
#control1sec.ipynb
#!!! the two leiden notebooks here